## Result 2: VSI
Vertical signal integrity (VSI) was accessed using `ovrl.py` (See `Ovrlpy_all_transcripts.ipynb`).  

This notebook focuses on the VSI map at Bregma = -0.24 and VSI differences between MOD1 and MOD2.

In [ ]:
import sys
import os
from pathlib import Path

sys.path.append(os.path.abspath("../src"))
from utils import load_merfish_signal_df, load_banksy_result, load_merfish_data, load_boundaries_data
from utils import extract_cell_vsi
from plot import plot_VSI_map, plot_vsi_with_named_squares, plot_vsi_distribution_comparison, plot_normalized_histogram, plot_vsi_qqplot

from IPython.display import display
import numpy as np
import pandas as pd

### data

#### Signals in the Tissue Section

In [ ]:
MERFISH_data_folder_path = Path("../data/mouse_hypothalamus/MERFISH/")

In [ ]:
signal_coordinate_df, coordinate_x_m, coordinate_y_m = load_merfish_signal_df(MERFISH_data_folder_path/"merfish_barcodes_example.csv")

#### Results of Ovrlpy

results:  
- signal integrity  
- signal strength  

In [ ]:
ovrlpy_result_folder = Path("../data/results/VSI")

In [ ]:
signal_integrity = np.loadtxt(ovrlpy_result_folder/"SignalIntegrity.txt")
signal_strength = np.loadtxt(ovrlpy_result_folder/"SignalStrength.txt")

#### Results of BANKSY

In [ ]:
banksy_folder_path = Path("../data/banksy_results/")

In [ ]:
banksy_result = load_banksy_result(banksy_folder_path/"banksy_cluster.txt", coordinate_x_m, coordinate_y_m)

#### Segmentation Dataset

In [ ]:
merfish_data = load_merfish_data(MERFISH_data_folder_path / "merfish_all_cells.csv", 
                                 banksy_result, coordinate_x_m, coordinate_y_m, 
                                 animal_id=1, bregma_value=-0.24)

#### Cell boundaries

In [ ]:
boundaries_df = load_boundaries_data(MERFISH_data_folder_path/'cellboundaries_example_animal.csv', 
                                     merfish_data, coordinate_x_m, coordinate_y_m)

#### MOD
BANKSY cluster:  
- 7: MMOD2 / MOD2  
- 8: MMOD1 / MOD1

In [ ]:
# boundaries, MOD
# MOD_boundaries = boundaries_df[(boundaries_df['banksy'] == 8) | (boundaries_df['banksy'] == 7)]
MOD1_boundaries = boundaries_df[boundaries_df['banksy'] == 8]
MOD2_boundaries = boundaries_df[boundaries_df['banksy'] == 7]

### VSI across the slice
Main findings:  
- Lower VSI at cell borders
- May cause false cell identification

#### overall

In [ ]:
SIM = plot_VSI_map(
    cell_integrity = signal_integrity,
    cell_strength = signal_strength,
    signal_threshold=3.0,
    figure_height=10/2.54,
    cmap="BIH",
    side_display="hist", # "hist", "colorbar", or None
    plot_boundary=False,
    plot_centroid=False
)

display(SIM)

In [ ]:
# VSI map without cell boundaries
SIM_noboundaries = plot_VSI_map(
    cell_integrity = signal_integrity,
    cell_strength = signal_strength,
    signal_threshold=3.0,
    figure_height=10/2.54,
    cmap="BIH",
    side_display="colorbar",
    plot_boundary=False,
    plot_centroid=False,
)

display(SIM_noboundaries)

#### cell level

In [ ]:
cell_integrity, cell_strength = extract_cell_vsi(
    boundary_df=boundaries_df, 
    integrity=signal_integrity,
    strength=signal_strength,
    integrity_size=1800
)

In [ ]:
cell_SIM_nb = plot_VSI_map(
    cell_integrity = cell_integrity,
    cell_strength = cell_strength,
    signal_threshold=3.0,
    figure_height=10/2.54,
    cmap="BIH",
    # side_display="hist",
    side_display="colorbar",
    plot_boundary=False,
    plot_centroid=False,
)

display(cell_SIM_nb)

In [ ]:
cell_SIM = plot_VSI_map(
    cell_integrity = cell_integrity,
    cell_strength = cell_strength,
    signal_threshold=3.0,
    figure_height=10/2.54,
    cmap="BIH",
    side_display=None,
    boundary_df=boundaries_df,
    plot_boundary=True,
    boundary_color="yellow",
    boundary_width=1.3,
    plot_centroid=False,
    scalebar=None
)

#### ROI for findings
ROIs show the low vsi at borders and false cell identification

In [ ]:
regions = [
    {"x": 400, "y": 1190, "name": "Region 1"},
    {"x": 1400, "y": 250, "name": "Region 4"},
    {"x": 50, "y": 440, "name": "Region 3"},
    {"x": 1350, "y": 950, "name": "Region 2"},
]

In [ ]:
VSI_ROI = plot_vsi_with_named_squares(cell_integrity, cell_strength, 
                                      signal_threshold=3.0,
    figure_height=10/2.54,
    cmap="BIH",
    side_display=None,
    # side_display='colorbar',
    boundary_df=boundaries_df,
    plot_boundary=False,
    boundary_color="yellow",
    boundary_width=0.3,
    plot_centroid=False,
    named_squares=regions)

display(VSI_ROI)

In [ ]:
# region 1

# Access the heatmap axis and set zoomed-in limits
ax = cell_SIM.axes[0]
ax.set_ylim(1190, 1290)
ax.set_xlim(400, 500)

cell_SIM

In [ ]:
# region 2

# Access the heatmap axis and set zoomed-in limits
ax = cell_SIM.axes[0]  
ax.set_ylim(250, 350)
ax.set_xlim(1400, 1500)

cell_SIM

In [ ]:
# region 3

# Access the heatmap axis and set zoomed-in limits
ax = cell_SIM.axes[0]  
ax.set_ylim(440, 540)
ax.set_xlim(50, 150)

cell_SIM

In [ ]:
# region 4

ax = cell_SIM.axes[0]  
ax.set_ylim(950, 1050)
ax.set_xlim(1350, 1450)

cell_SIM

### VSI in MODs
compare the VSI values in MOD1 and MOD2  
- MOD2 shows lower VSI

#### MOD1 vs. MOD2

In [ ]:
MOD1_int, MOD1_str = extract_cell_vsi(MOD1_boundaries, signal_integrity, signal_strength)
MOD2_int, MOD2_str = extract_cell_vsi(MOD2_boundaries, signal_integrity, signal_strength)

In [ ]:
vals1, bins1, vals2, bins2 = plot_vsi_distribution_comparison(
    cell_integrity_1=MOD1_int,
    cell_strength_1=MOD1_str,
    cell_integrity_2=MOD2_int,
    cell_strength_2=MOD2_str,
    signal_threshold=3.0,
    figure_height=6/2.54,
    cmap="BIH",
    ylim=(1e-1,64)
)

In [ ]:
plot_normalized_histogram(vals1, vals2, bins=bins1, epsilon= 1e-10, ylim=(1e-1, 10**11))

In [ ]:
plot_vsi_qqplot(vals1, bins1, vals2, bins2)